In [13]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, Subset, WeightedRandomSampler
from torch.cuda.amp import GradScaler, autocast

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scaler = GradScaler()

C:\Users\FireFly\AppData\Local\Temp\ipykernel_10204\650832284.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [15]:
DATA_DIR     = r"D:\res_work\ECG_analysis_for_CVD\processed_images_green"
NUM_CLASSES  = 4
BATCH_SIZE   = 8       # per-GPU batch
ACCUM_STEPS  = 4       # effective batch = 32
TOTAL_EPOCHS = 50
LR_HEAD      = 1e-3
LR_FEAT      = 1e-4
WEIGHT_DECAY = 1e-4
TRAIN_RATIO  = 0.7
TEST_RATIO   = 0.2
VAL_RATIO    = 0.1
PATIENCE     = 10      # early stopping patience
DROP_PROB    = 0.5      # dropout probability
MIXUP_ALPHA  = 0.4      # mixup alpha
SMOOTHING    = 0.1      # label smoothing

In [16]:
train_tf = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomResizedCrop(224, scale=(0.8,1.0)),
    transforms.RandAugment(num_ops=2, magnitude=9),
    transforms.ColorJitter(0.2,0.2,0.2,0.1),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.5, scale=(0.02,0.15), ratio=(0.3,3.3)),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225]),
])
val_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225]),
])

In [17]:
full_ds = datasets.ImageFolder(DATA_DIR)
N = len(full_ds)
n_train = int(TRAIN_RATIO * N)
n_test  = int(TEST_RATIO  * N)
n_val   = N - n_train - n_test

g = torch.Generator().manual_seed(42)
perm = torch.randperm(N, generator=g).tolist()
train_idx = perm[:n_train]
test_idx  = perm[n_train:n_train+n_test]
val_idx   = perm[n_train+n_test:]

train_targets = [full_ds.targets[i] for i in train_idx]
class_counts  = np.bincount(train_targets, minlength=NUM_CLASSES)
class_weights = 1. / class_counts
sample_weights = [class_weights[t] for t in train_targets]
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

train_ds = Subset(datasets.ImageFolder(DATA_DIR, transform=train_tf), train_idx)
val_ds   = Subset(datasets.ImageFolder(DATA_DIR, transform=val_tf),   val_idx)
test_ds  = Subset(datasets.ImageFolder(DATA_DIR, transform=val_tf),   test_idx)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler, num_workers=4)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False,     num_workers=4)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False,     num_workers=4)

In [18]:
base_model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
# Freeze early layers
for p in base_model.features[:16].parameters(): p.requires_grad = False
# Unfreeze last conv block
for p in base_model.features[16:].parameters(): p.requires_grad = True

# Replace classifier head
in_features = base_model.classifier[0].in_features
base_model.classifier = nn.Sequential(
    nn.Dropout(DROP_PROB),
    nn.Linear(in_features, 512),
    nn.ReLU(inplace=True),
    nn.Dropout(DROP_PROB),
    nn.Linear(512, 256),
    nn.ReLU(inplace=True),
    nn.Dropout(DROP_PROB),
    nn.Linear(256, NUM_CLASSES)
)
model = base_model.to(device)

In [20]:
def mixup_data(x, y, alpha=MIXUP_ALPHA):
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    return mixed_x, y, y[index], lam

def mixup_criterion(crit, pred, y_a, y_b, lam):
    return lam * crit(pred, y_a) + (1 - lam) * crit(pred, y_b)

criterion = nn.CrossEntropyLoss(label_smoothing=SMOOTHING)
optimizer = optim.AdamW([
    {'params': model.features[16:].parameters(), 'lr': LR_FEAT},
    {'params': model.classifier.parameters(),    'lr': LR_HEAD}
], weight_decay=WEIGHT_DECAY)
scheduler = OneCycleLR(
    optimizer,
    max_lr=[LR_FEAT, LR_HEAD],
    total_steps=TOTAL_EPOCHS * len(train_loader),
    pct_start=0.1,
    anneal_strategy='cos'
)
class EarlyStopping:
    def __init__(self, patience=PATIENCE):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False
    def __call__(self, val_acc):
        if self.best_score is None or val_acc > self.best_score:
            self.best_score = val_acc
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

early_stopper = EarlyStopping()

In [22]:
best_val_acc = 0.0
for epoch in range(1, TOTAL_EPOCHS+1):
    model.train()
    running_loss = running_corr = 0
    optimizer.zero_grad()
    for i, (imgs, labels) in enumerate(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        mixed_imgs, y_a, y_b, lam = mixup_data(imgs, labels)
        with autocast():
            outputs = model(mixed_imgs)
            loss = mixup_criterion(criterion, outputs, y_a, y_b, lam) / ACCUM_STEPS
        scaler.scale(loss).backward()
        if (i+1) % ACCUM_STEPS == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        scheduler.step()

        running_loss += loss.item() * ACCUM_STEPS
        preds = outputs.argmax(1)
        running_corr += (lam * (preds==labels).sum().item() +
                         (1-lam) * (preds==labels[torch.randperm(labels.size(0)).to(device)]).sum().item())

    train_loss = running_loss / n_train
    train_acc  = running_corr / n_train * 100

    # validation
    model.eval()
    val_loss = val_corr = 0
    with torch.no_grad(), autocast():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            val_loss += criterion(outputs, labels).item()
            val_corr += (outputs.argmax(1)==labels).sum().item()
    val_loss /= n_val
    val_acc  = val_corr / n_val * 100

    early_stopper(val_acc)
    print(f"Epoch {epoch}/{TOTAL_EPOCHS} | Train loss={train_loss:.4f}, acc={train_acc:.2f}% | "
          f"Val loss={val_loss:.4f}, acc={val_acc:.2f}%")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_vgg16_ecg_processed_green.pth")
        print(" → New best model saved!")
    if early_stopper.early_stop:
        print(f"Early stopping at epoch {epoch}")
        break

print(f"Training complete. Best validation accuracy: {best_val_acc:.2f}%")

C:\Users\FireFly\AppData\Local\Temp\ipykernel_10204\3528197656.py:9: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
C:\Users\FireFly\AppData\Local\Temp\ipykernel_10204\3528197656.py:30: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), autocast():


Epoch 1/50 | Train loss=0.1739, acc=28.85% | Val loss=0.1670, acc=52.13%
 → New best model saved!
Epoch 2/50 | Train loss=0.1612, acc=34.57% | Val loss=0.1383, acc=61.70%
 → New best model saved!
Epoch 3/50 | Train loss=0.1482, acc=41.35% | Val loss=0.1070, acc=76.60%
 → New best model saved!
Epoch 4/50 | Train loss=0.1366, acc=47.33% | Val loss=0.0937, acc=85.11%
 → New best model saved!
Epoch 5/50 | Train loss=0.1469, acc=40.87% | Val loss=0.1169, acc=68.09%
Epoch 6/50 | Train loss=0.1293, acc=47.25% | Val loss=0.0847, acc=94.68%
 → New best model saved!
Epoch 7/50 | Train loss=0.1210, acc=49.46% | Val loss=0.0929, acc=86.17%
Epoch 8/50 | Train loss=0.1224, acc=53.05% | Val loss=0.0780, acc=89.36%
Epoch 9/50 | Train loss=0.1164, acc=52.77% | Val loss=0.0710, acc=92.55%
Epoch 10/50 | Train loss=0.1156, acc=55.38% | Val loss=0.0842, acc=82.98%
Epoch 11/50 | Train loss=0.1179, acc=54.19% | Val loss=0.0727, acc=92.55%
Epoch 12/50 | Train loss=0.1107, acc=56.52% | Val loss=0.0761, acc=91.

In [23]:
model.load_state_dict(torch.load("best_vgg16_ecg_processed_green.pth"))
model.eval()
test_corr = 0
with torch.no_grad(), autocast():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        test_corr += (outputs.argmax(1)==labels).sum().item()
test_acc = test_corr / n_test * 100
print(f"Test Accuracy: {test_acc:.2f}%")

C:\Users\FireFly\AppData\Local\Temp\ipykernel_10204\4110255428.py:4: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), autocast():


Test Accuracy: 97.84%
